<a href="https://colab.research.google.com/github/jefftwebb/MSBA-Capstone/blob/main/EDA_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA with  SQL, Python and R in Google Colab



## 1. Connect to BigQuery and format table display.

This interactive authentication requires that you have a google account.

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [6]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

This feature makes it more convenient to navigate tables within the notebook.

## 2. Magic commands to write SQL and R code



First: what are magic commands? 

In [ ]:
%magic

Which ones are available?

In [ ]:
%lsmagic

Notice that there are two levels of magic commands:

- %% affects the entire cell; 
- % affects an individual line.

In this tutorial we'll be using bigquery magics (to write SQL against tables in bigquery) and R magics (to write R code).

For example, the following cell uses %%bigquery to query the google analytics public dataset, storing the result in a pandas dataframe (the default format in .ipynb) defined in the cell magic statement as "df." Notice that I have referenced my own Google cloud project in the magic command. **You will need to change the project identifier to point to your own GCP project.**

Here for illustration we return 10 rows from the entire dataset for Aug 1, 2017.

In [2]:
%%bigquery --project capstone-359314 df

SELECT * FROM bigquery-public-data.google_analytics_sample.ga_sessions_20170801
limit 10



In [ ]:
type(df)

In [ ]:
df.shape

In [ ]:
df.info()


In [ ]:
df.head()

The arrays from BigQuery have been brought into a pandas dataframe as what appear to be JSON columns. These include: totals, trafficSource, device, geoNetwork, and hits.

I prefer to do EDA in R, so my ultimate goal is to produce a rectangular dataset that can be read into R for data exploration. However R will not accept the JSON columns (which it interprets as python dicts). To prepare for EDA in R  let's go back and revise the initial query to flatten the dataset (as well as to pull in more dates) with one row per visit.

## 3. Flatten Data

For reference, here is a detailed [data dictionary](https://support.google.com/analytics/answer/3437719?hl=en&ref_topic=3416089).

You will most likely want to create different tables as you explore different ideas during EDA.

For this exercise, we will focus on a few tables and bring in a weeks's worth of data. (The query is still time-consuming.)



In [18]:
%%bigquery --project capstone-359314 df 

SELECT fullVisitorId, visitId, date,  device.*, totals.*, channelGrouping
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20160808'



In [ ]:
df.shape

In [ ]:
df.columns

## 4. Import data into R.

Load the rpy2 extension.

In [ ]:
%load_ext rpy2.ipython


Use R magic to load the df dataset into R. (-i stands for "input.")

In [22]:
%R -i df

Load the tidyverse packages.



In [ ]:
%R library(tidyverse)

Inspect the data.

In [ ]:
%R str(df)

Looks good!

## 5. EDA

EDA is an art and a science. There is not one right way to explore a dataset. Use your creativity and critical thinking to ask and answer questions, perhaps starting with some QA and then increasingly focusing on the target variable and possible predictors.

Let's assume for now that the target variable for this project will be `transactionRevenue`.

It can be helpful at the beginning of the EDA process to list a set of questions to be investigated. 

- what is the distribution of `transactionRevenue`?
- are there invariant columns that should be removed?
- what is the scope of missing data?
- are there daily patterns of site visits?
- are certain devices more correlated with purchasing than others?

Etc.



### Distribution of `transactionRevenue` 

In [ ]:
%R head(df$transactionRevenue, 1000)

Clearly, a session that did not result in a purchase is coded `NaN`.  That should be recoded as 0 and the amount of the purchase should be returned to a legible number by dividing by 10^6. While we're at it, we will copy this column and shorten the name.

In [28]:
%%R 

df <- df %>% 
mutate(rev = transactionRevenue,
       rev = rev/10^6,
       rev = replace_na(rev, 0))

In [ ]:
%R head(df$rev, 1000)

What is the percentage of visits that result in purchases?

In [ ]:
%R sum(df$rev > 0)/nrow(df)*100

How many purchases is that exactly?

In [ ]:
%R sum(df$rev > 0)

Hmmm. Not many. 169 out of 18,704 visits.

What is the distribution of purchases?

In [ ]:
%%R

df %>%
filter(rev > 0) %>%
ggplot(aes(rev)) +
geom_histogram() +
labs(title = "Distribution of purchase amounts") +
theme_minimal()


In [ ]:
%%R

df %>%
filter(rev > 0) %>%
select(rev) %>%
summary


Long right tail. 

Log normal? Looks like it:

In [ ]:
%%R

df %>%
filter(rev > 0) %>%
ggplot(aes(log(rev))) +
geom_histogram() +
labs(title = "Distribution of log purchase amounts") +
theme_minimal()

### Identify and remove invariant columns



In [ ]:
%%R 

apply(df, 2, n_distinct)

In [ ]:
%%R 

apply(df, 2, n_distinct) > 1

No use this boolean to select columns.

In [40]:
%%R

df <- df %>%
select_if(apply(df, 2, n_distinct) >1)

### What is the scope of missing data?

In [ ]:
%%R 

summary(df)

### Are purchases more likely with mobile?

No, not really:

In [ ]:
%%R

df %>% 
group_by(isMobile) %>%
summarize(prop_purchase = sum(rev > 0)/n())

### Are purchases related to the channel?

Referral, though small in absolute terms, accounts for the largest proportion of purchases. Mobile is only involved in organic search and direct.

In [ ]:
%%R

df %>% 
group_by(channelGrouping, isMobile) %>%
summarize(prop_purchase = sum(rev > 0)/n()) %>%
ggplot(aes(channelGrouping, prop_purchase, fill = isMobile)) +
geom_col(position = "dodge") +
theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))

### What is the distribution of pageviews?

In [ ]:
%%R 

df %>%
ggplot(aes(pageviews)) +
geom_histogram()


Related to purchasing? Specifically does the proportion of purchases go up with bucketed page views?

In [ ]:
%%R 

df %>%
filter(pageviews > 1) %>%
mutate(pageview_bins = cut(pageviews, 15)) %>%
group_by(pageview_bins) %>%
summarize(prop_purchase = sum(rev > 0)/n(),
          n()) 